# Preposseing of the crsp data 

In [14]:
# Load libraries
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from datetime import datetime
import pandas as pd
from urllib.request import urlopen

In [32]:
# Read CSV data from URL
df = pd.read_csv("/Users/jensknudsen/Desktop/seminar/datashare.csv")

# make a new column with the date as a datetime object
df['datetime'] = pd.to_datetime(df['DATE'].astype(str), format='%Y%m%d')

# make a function that findt the first day of the closest month
def get_closest_first_day(date):
    year = date.year
    month = date.month
    day = date.day
    if day > 15:
        if month == 12:
            month = 1
            year += 1
        else:
            month += 1
    return datetime.datetime(year, month, 1)

# apply the function to the dataframe
df['month'] = df['datetime'].apply(get_closest_first_day)

# delete the not used date columns
del df["DATE"]
del df["datetime"]

# make chareacteristic variables
df = df.rename(columns={col: f"characteristic_{col}" for col in df.columns if col not in ["permno", "month", "sic2"]})

permno  characteristic_mvel1  characteristic_beta  \
0         10006          8.630850e+04             1.117907   
1         10014          3.903375e+03             0.331304   
2         10022          9.841000e+03             0.942052   
3         10030          5.190100e+04             0.886532   
4         10057          3.525000e+04             1.229520   
...         ...                   ...                  ...   
3760203   93428          1.250976e+06             1.595315   
3760204   93429          5.600537e+06             0.298267   
3760205   93433          2.473625e+04             2.324893   
3760206   93434          8.573280e+04             0.628519   
3760207   93436          2.840318e+07             1.588092   

         characteristic_betasq  characteristic_chmom  characteristic_dolvol  \
0                     1.249717              0.134574              10.296745   
1                     0.109762              0.006667               7.032404   
2                     0.887461              0.058761               7.294038   
3                     0.785940             -0.032065               9.516942   
4                     1.511719             -0.102811               8.421013   
...                        ...                   ...                    ...   
3760203               2.545028             -0.137348              14.916210   
3760204               0.088963              0.206434              16.319190   
3760205               5.405128              0.328248               9.975208   
3760206               0.395036              0.020854              10.862196   
3760207               2.522036             -0.121017              18.580861   

         characteristic_idiovol  characteristic_indmom  characteristic_mom1m  \
0                      0.024863               0.059540              0.002016   
1                      0.065248               0.059540             -0.086957   
2                      0.029338               0.059540             -0.037037   
3                      0.022007               0.059540              0.000000   
4                      0.025453               0.059540             -0.030717   
...                         ...                    ...                   ...   
3760203                0.043201               0.016401             -0.001203   
3760204                0.026198              -0.071728              0.093973   
3760205                0.185537              -0.002451              0.279070   
3760206                0.059797               0.015187             -0.049505   
3760207                0.049922               0.024821             -0.042128   

         characteristic_mom6m  ...  characteristic_ms  \
0                    0.081150  ...                NaN   
1                   -0.080000  ...                NaN   
2                   -0.005452  ...                NaN   
3                    0.033590  ...                NaN   
4                    0.048778  ...                NaN   
...                       ...  ...                ...   
3760203             -0.047018  ...                6.0   
3760204              0.000143  ...                6.0   
3760205             -0.358209  ...                2.0   
3760206              0.109890  ...                3.0   
3760207             -0.114232  ...                4.0   

         characteristic_baspread  characteristic_ill  characteristic_maxret  \
0                       0.013856        1.557681e-07               0.029167   
1                       0.031389        8.383815e-06               0.071429   
2                       0.010066        5.360386e-06               0.020000   
3                       0.009523        1.090908e-07               0.017857   
4                       0.007661        2.386914e-07               0.007194   
...                          ...                 ...                    ...   
3760203                 0.029302        1.010214e-09               0.027604   
3760204                 0.016001     

# Load the ML file from the beginning

In [6]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import sqlite3
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Print all the vairables in the environment
connection = sqlite3.connect("/Users/jensknudsen/Desktop/Seminar/tidy_finance_ML.sqlite")
connection.row_factory = sqlite3.Row
cursor = connection.execute('select * from stock_characteristics_monthly')
# instead of cursor.description:
row = cursor.fetchone()
names = row.keys()
print(names)

['permno', 'month', 'ret_excess', 'mktcap_lag', 'sic2', 'macro_dp', 'macro_dy', 'macro_ep', 'macro_de', 'macro_svar', 'macro_bm', 'macro_ntis', 'macro_tbl', 'macro_lty', 'macro_ltr', 'macro_tms', 'macro_dfy', 'macro_infl', 'characteristic_mvel1', 'characteristic_beta', 'characteristic_betasq', 'characteristic_chmom', 'characteristic_dolvol', 'characteristic_idiovol', 'characteristic_indmom', 'characteristic_mom1m', 'characteristic_mom6m', 'characteristic_mom12m', 'characteristic_mom36m', 'characteristic_pricedelay', 'characteristic_turn', 'characteristic_absacc', 'characteristic_acc', 'characteristic_age', 'characteristic_agr', 'characteristic_bm', 'characteristic_bm_ia', 'characteristic_cashdebt', 'characteristic_cashpr', 'characteristic_cfp', 'characteristic_cfp_ia', 'characteristic_chatoia', 'characteristic_chcsho', 'characteristic_chempia', 'characteristic_chinv', 'characteristic_chpmia', 'characteristic_convind', 'characteristic_currat', 'characteristic_depr', 'characteristic_divi

In [3]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("/Users/jensknudsen/Desktop/Seminar/tidy_finance_ML.sqlite")

# Create a pandas DataFrame from the "crsp_monthly" table in the SQLite database
crsp_monthly = pd.read_sql_query("SELECT permno, month, ret_excess, mktcap_lag, sic2, macro_dp, macro_bm, macro_ntis, macro_tbl, characteristic_mom1m, characteristic_mvel1, characteristic_mom12m, characteristic_chmom, characteristic_maxret FROM stock_characteristics_monthly", conn)

# define a function to convert the number of days to a datetime object
def convert_days_to_datetime(num_days):
    start_date = datetime.datetime(1960, 1, 1)
    delta = datetime.timedelta(days=num_days)
    return start_date + delta

# apply the function to the 'date' column to convert it to datetime
crsp_monthly['month'] = crsp_monthly['month'].apply(convert_days_to_datetime)
crsp_monthly

# Select only the stocks that have 708 observations
df = crsp_monthly.query('month >= "1960-01-01"') \
                   .groupby('permno') \
                   .filter(lambda x: len(x) >= 611)

# Count the number of unique stocks in the DataFrame
num_stocks = len(df['permno'].unique())

# make a function that findt the first day of the closest month
def get_closest_first_day(date):
    year = date.year
    month = date.month
    day = date.day
    if day > 15:
        if month == 12:
            month = 1
            year += 1
        else:
            month += 1
    return datetime.datetime(year, month, 1)

# apply the function to the dataframe
df['date'] = df['month'].apply(get_closest_first_day)

In [14]:
# Increase the size of the heatmap.
plt.figure(figsize=(12, 8))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':1}, pad=13);

/var/folders/73/n7lgj33x23g9bs4jy5z1pgg40000gn/T/ipykernel_46322/1144810537.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)


In [37]:
# install matplotlib
import matplotlib.pyplot as plt

# plot the distribution of the variable characteristic_mvel1
df['characteristic_mvel1'].plot.hist(bins=100)

<Axes: ylabel='Frequency'>

# Example of LSTM

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np